In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
webseries = pd.read_csv('All_Streaming_Shows.csv')

In [3]:
# Cleanning Dataset
# series Title
# Conternt Rating
# Genre
# Descripition
# Streaming Platform
webseries = webseries[['SeriesTitle','Genre','Description','StreamingPlatform']]

In [4]:
webseries['Description'] = webseries['Description'].apply(lambda x:x.split())

In [5]:
webseries['Genre'] = webseries['Genre'].apply(lambda x:x.split())

In [6]:
webseries['StreamingPlatform'] = webseries['StreamingPlatform'].astype(str)

# Remove spaces from the values
webseries['StreamingPlatform'] = webseries['StreamingPlatform'].apply(lambda x: x.replace(" ", ""))
webseries['StreamingPlatform'] = webseries['StreamingPlatform'].apply(lambda x:x.split())

In [7]:
webseries['tags'] =  webseries['Genre'] + webseries['Description'] + webseries['StreamingPlatform']

In [8]:
webseries.head()

,SeriesTitle,Genre,Description,StreamingPlatform,tags
0,Breaking Bad,"[Crime,Drama]","[When, Walter, White,, a, New, Mexico, chemist...",[Netflix],"[Crime,Drama, When, Walter, White,, a, New, Me..."
1,Game of Thrones,"[Action, &, Adventure,Drama]","[Seven, noble, families, fight, for, control, ...","[HBOMAX,HBO]","[Action, &, Adventure,Drama, Seven, noble, fam..."
2,Rick and Morty,"[Animation,Comedy]","[Rick, is, a, mentally-unbalanced, but, scient...","[FreeServices,HBOMAX,Hulu]","[Animation,Comedy, Rick, is, a, mentally-unbal..."
3,Stranger Things,"[Drama,Fantasy]","[When, a, young, boy, vanishes,, a, small, tow...",[Netflix],"[Drama,Fantasy, When, a, young, boy, vanishes,..."
4,The Boys,"[Action, &, Adventure,Comedy]","[A, group, of, vigilantes, known, informally, ...",[PrimeVideo],"[Action, &, Adventure,Comedy, A, group, of, vi..."


In [9]:
new_df = webseries[['SeriesTitle','tags']]

In [10]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\ankit\AppData\Local\Temp\ipykernel_10572\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [11]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\ankit\AppData\Local\Temp\ipykernel_10572\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [12]:
new_df

,SeriesTitle,tags
0,Breaking Bad,"crime,drama when walter white, a new mexico ch..."
1,Game of Thrones,"action & adventure,drama seven noble families ..."
2,Rick and Morty,"animation,comedy rick is a mentally-unbalanced..."
3,Stranger Things,"drama,fantasy when a young boy vanishes, a sma..."
4,The Boys,"action & adventure,comedy a group of vigilante..."
...,...,...
12348,A Fishing Story with Ronnie Green,"2017,prime video a fishing story with ronnie g..."
12349,CMT Most Shocking,-1 -1 -1
12350,NHL Road to the Outdoor Classics,"2016,prime video road to the nhl outdoor class..."
12351,Addy Media,"2018,prime video addy media has one or more ep..."


In [13]:
import nltk

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [16]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\ankit\AppData\Local\Temp\ipykernel_10572\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=12000,stop_words='english')

In [18]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [19]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
cv.get_feature_names_out()

array(['00', '000', '001', ..., 'أوجيني', '潘惠美', '부릉'], dtype=object)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity = cosine_similarity(vectors)

In [23]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(116, 0.6779669064975021),
 (692, 0.6674844956390601),
 (141, 0.6298438669252062),
 (101, 0.6254017837577625),
 (7, 0.624436085122555)]

In [24]:
def recommend(web):
    movie_index = new_df[new_df['SeriesTitle'] == web].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].SeriesTitle)

In [25]:
recommend('Breaking Bad')

Broadchurch
American Crime
YOU
American Crime Story
Sherlock


In [26]:
import pickle

In [27]:
new_df['SeriesTitle'].values

array(['Breaking Bad', 'Game of Thrones', 'Rick and Morty', ...,
       'NHL Road to the Outdoor Classics', 'Addy Media',
       'My Dream Derelict Home'], dtype=object)

In [28]:
pickle.dump(new_df.to_dict(),open('web_dict.pkl','wb'))

In [29]:
pickle.dump(similarity,open('similaritys.pkl','wb'))